In [1]:
import xarray as xr
import numpy as np
import requests
import io

### Setup

In [2]:
def test_ensemble_mean(ds, lat, lon, models):

    for var in list(ds.data_vars):
        if var not in ["spatial_ref", "crs"]:
            print(f"Indicator: {var}")

            print("Original Dataset Values:")
            values = (
                ds[var]
                .sel(
                    lat=lat,
                    lon=lon,
                    method="nearest",
                )
                .sel(model=models)
                .values
            )

            print(values)
            print("\n Calculated Ensemble Mean:")
            # skip -9999 when calculating the mean
            values = np.where(values == -9999, np.nan, values)
            print(np.nanmean(values))

            print("Dataset Ensemble Mean:")
            print(
                ds[var]
                .sel(
                    lat=lat,
                    lon=lon,
                    method="nearest",
                )
                .sel(model=13)
                .values
            )

            print("\n\n")

In [12]:
# test will use this location
lat = 65
lon = -142
models = slice(0, 13)  # first 12 models - model 13 is the ensemble mean

# query only first and last timesteps / scenarios of the datacube
times = [14, 55098.5]
scenarios = [0, 4]

hist_url = f"https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=cmip6_monthly_cf&SUBSET=model(0)&SUBSET=lat(64,65)&SUBSET=lon(-145,-144)&SUBSET=scenario({scenarios[0]})&FORMAT=application/netcdf"
proj_url = f"https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=cmip6_monthly_cf&SUBSET=model(0)&SUBSET=lat(64,65)&SUBSET=lon(-145,-144)&SUBSET=scenario({scenarios[1]})&FORMAT=application/netcdf"

### Historical

In [13]:
with requests.get(hist_url) as response:
    print(hist_url)
    ds_hist = xr.open_dataset(
        io.BytesIO(response.content), decode_cf=True, mask_and_scale=False
    )
ds_hist

https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=cmip6_monthly_cf&SUBSET=model(0)&SUBSET=lat(64,65)&SUBSET=lon(-145,-144)&SUBSET=scenario(0)&FORMAT=application/netcdf


ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'cfgrib', 'pydap', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html